# Switching to Pre-Mixed Correlated K-Tables

We are currently working on building out the k-table functionality. Currently, the only available functionality is to use the pre-mixed correlated k-tables provided by our group. You can find these k-tables on Zenodo and details are included in Marley et al. 2020.  


In [8]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import astropy.units as u

#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi
#plotting
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import show

Loading BokehJS ...

In [9]:
#let's do two calculations 
#first, the regular monochromatic opacities
opacity_mono = jdi.opannection(wave_range=[1,14]) #lets just use all defaults


Notice that when we call the `opannection` with the `ck=True`, we must also pass it one of the directories from Zenodo. The directory name describes the chemistry that this has been calculated for. In this case it is 1xSolar, Solar C/O. 

**Inside that directory there should be at least two files: ascii_data, and full_abunds**


In [10]:
#second the correlated k table
opacity_ck = jdi.opannection(ck=True, 
                             ck_db='m+0.0_co0.5.data.196'
                            ) 


In [11]:
#lets use the same example Jupiter PT profile
pt= pd.read_csv(jdi.jupiter_pt(), delim_whitespace=True,usecols=[0,1])

You should already be familiar with the following code block. Here we are loading the input class, setting the gravity, stellar parameters, and sonora profile. 

In [5]:
calc = {'ck':opacity_ck,'mono':opacity_mono}
cases = {i: jdi.inputs() for i in calc.keys()}
cases['mono'].approx(raman='none')
#phase angle 
for i in calc.keys(): cases[i].phase_angle(0) #radians

#define gravity
for i in calc.keys(): cases[i].gravity(radius=1, 
                                    radius_unit=u.Unit('R_jup'),
                                    mass=1,
                                    mass_unit=u.Unit('M_jup')) #any astropy units available

#define star 
for i in calc.keys(): cases[i].star(calc[i], 5000,0,4.0,
                                   radius=1,
                                   radius_unit=u.Unit('R_sun'),
                                   semi_major=5,
                                   semi_major_unit=u.Unit('au')) #opacity db, pysynphot database, temp, metallicity, logg 

sonora_profile_db = '/Users/nbatalh1/Documents/data/sonora_profile/'
#just grabbing any Teff so we can use a sonora pressure-temperature profile
Teff = 1500
for i in calc.keys():cases[i].sonora(sonora_profile_db, Teff)

The differences between the two procedures comes in when we specify the chemistry. Here we introduce a new function called `premix_atmosphere`. This is going to pull the precomputed chemistry that was used to compute the correlated k tables. 

In [12]:
    
cases['ck'].premix_atmosphere(opacity_ck, 
            df = cases['ck'].inputs['atmosphere']['profile'].loc[:,['pressure','temperature']])

#now let's pass that to the monochromatic opacities for consistency
cases['mono'].atmosphere(df=cases['ck'].inputs['atmosphere']['profile'])


In [13]:
df = {}
for i in calc.keys(): 
    df[i] = cases[i].spectrum(calc[i],full_output=True,
                              calculation='thermal')

## Compare Monochromatic Opacities with Pre-Mixed K-Tables
 

In [14]:
compare = 'thermal'
wno_ck, therm_ck = df['ck']['wavenumber'] , df['ck'][compare]
wno_mon1, therm_mon1 = jdi.mean_regrid(df['mono']['wavenumber'], 
                                       df['mono'][compare] , newx=wno_ck)
wno_mon2, therm_mon2 = jdi.mean_regrid(df['mono']['wavenumber'], 
                                       df['mono'][compare] , R=200)

In [16]:
show(jpi.spectrum([wno_ck,wno_mon1], 
                  [therm_ck,therm_mon1],
                  legend=['monochromatic','k table'], plot_width=500,
                  y_axis_type='log'
                 ,x_axis_type='log'))